# 모둘화됨

In [1]:
from accounts import jp
# jp.ID, jp.PW

In [2]:
import os
from datetime import datetime
import time
import requests
import re

import pandas as pd

from bs4 import BeautifulSoup

import selenium

from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [22]:
############################
# 크롬으로 잡플래닛 사이트 열기
############################

service = Service(executable_path="chromedriver.exe")
browser = webdriver.Chrome(service=service)

url = r"https://www.jobplanet.co.kr/job"

browser.maximize_window()
print(browser.get_window_size())

browser.get(r"https://www.jobplanet.co.kr/job")

{'width': 1512, 'height': 944}


In [23]:
############################
# 잡플래닛에 로그인 하기
############################

# 로그인 버튼 찾기 및 클릭
login_btn = browser.find_element(By.CSS_SELECTOR,"a.btn_txt.login")
login_btn.click()

# 로그인 ID와 PW 쳐야할 곳 클릭 + 적기
login_Id = browser.find_element(By.CSS_SELECTOR, "#user_email")
login_Id.click()
login_Id.send_keys(jp.ID)

login_Pw = browser.find_element(By.CSS_SELECTOR, "#user_password")
login_Pw.click()
login_Pw.send_keys(jp.PW)

# 이메일로 로그인 버튼 누르기
em_log = browser.find_element(By.CSS_SELECTOR, "fieldset > button")
em_log.click()

In [5]:
# 모둘화됨

In [24]:
###########################
# 잡플래닛 검색창에 회사 입력하고 맨 처음에 나오는 기업 클릭하기.
###########################
# 검색창 누르기
search_bar = browser.find_element(By.CSS_SELECTOR, "#search_bar_search_query")
search_bar.click()

# 내가 원하는 회사 입력하기
want_cp = input()
search_bar.send_keys(want_cp)
search_bar.send_keys(Keys.RETURN)

# 검색결과 기다리기.
time.sleep(1)

# 맨처음에 나오는 기업 클릭하기
company = browser.find_element(By.CSS_SELECTOR, "div.is_company_card > div:nth-child(1) > a")
company.click()

In [25]:
###########################
# 회사페이지에서 리뷰 버튼 누르기
###########################
# 팝업창이 있을 때는 팝업창 버튼을 누르고 리뷰버튼을 누르기
try:
    pop = browser.find_element(By.CSS_SELECTOR, "div.premium_modal_header > button")
    pop.click()
    review_bar = browser.find_element(By.CSS_SELECTOR, "li.viewReviews > a")
    review_bar.click()

except NoSuchElementException:
    # 만약 팝업창 버튼이 없다면 리뷰 버튼만 누르기
    try:
        review_bar = browser.find_element(By.CSS_SELECTOR, "li.viewReviews > a")
        review_bar.click()
        time.sleep(1)
        pop = browser.find_element(By.CSS_SELECTOR, "div.premium_modal_header > button")
        pop.click()
    except NoSuchElementException:
        pass
    pass

In [26]:
###########################
# 회사페이지에서 리뷰 가져오기.
###########################
def get_jp_reviews():
    """
    잡플래닛 회사 리뷰 상세 내용을 url을 받아서 내용을 반환.
    연결
    """
    # 리뷰들을 리스트로 저장하는 변수
    good_review_list = []
    bad_review_list = []

    # 이전페이지 변수
    pre_page = 0

    # 무한 루프를 돌기.
    while True:

        # 소스 끌어오기.
        html = browser.page_source
        soup = BeautifulSoup(html, "lxml")

        # 현재 페이지의 숫자를 가져오기.
        crt_btn = soup.select_one('strong.txtlink_page')
        crt_page = int(crt_btn.text.strip())

        # 다음버튼을 눌렀은데도 같은 페이지인 경우
        if crt_page == pre_page:
            break

        # html 내용 가져오기
        good_review = soup.select("section > div > div.ctbody_col2 > div > dl > dd:nth-child(2) > span")
        bad_review = soup.select("section > div > div.ctbody_col2 > div > dl > dd:nth-child(4) > span")

        # 텍스트로 추출한 내용을 리스트에 추가해서 넣어주기 위해 이용.
        for good_text in good_review: # for문은 for 채워져있는 값의 원소 in 채워져있는 리스트
            # 데이터 전처리
            good_text_clean_1 = good_text.get_text().strip()
            good_text_clean_2 = good_text_clean_1.replace("\n", " ")

            # 장점 리뷰에 넣기.
            good_review_list.append(good_text_clean_2)

        for bad_text in bad_review:
            # 데이터 전처리
            bad_text_clean_1 = bad_text.get_text().strip()
            bad_text_clean_2 = bad_text_clean_1.replace("\n", " ")

            # 단점 리뷰에 넣기.
            bad_review_list.append(bad_text_clean_2)

        time.sleep(1)

        # 다음 페이지로 이동하기 위한 버튼을 찾기.
        try:
            # 이전페이지의 번호를 갱신하기.
            pre_page = crt_page

            # 다음 버튼 찾기
            next_btn = browser.find_element(By.CSS_SELECTOR,f"a.btn_pgnext")

            # 버튼이 있으면 클릭합니다.
            next_btn.click()

            # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
            time.sleep(3)

        except:
            break

        # local에 디렉토리를 생성.
        save_path = f"/Users/gyeonmunju/Desktop/PlayData/jp_company"
        os.makedirs(save_path, exist_ok=True)

        # 데이터프레임으로 관리하겠다 지정. (데이터 생성하기.)
        good_df = pd.DataFrame({"긍정적 리뷰" : good_review_list})
        bad_df = pd.DataFrame({"부정적 리뷰" : bad_review_list})
        # 긍정적/부정적 리뷰 합치기
        jp_df = good_df.join(bad_df)

        # csv 파일로 저장.
        file_name = f"{want_cp}reviews.csv"
        save_file_path = os.path.join(save_path, file_name)
        jp_df.to_csv(save_file_path, index=False, encoding = "utf-8")

        print(len(jp_df.index))
        # 데이터프레임의 인덱스 개수 확인 후 60개를 초과하면 다음 코드로 넘어가기.
        if len(jp_df.index) > 60:
            break

    return jp_df

# 함수 호출 후 결과 출력
get_jp_reviews()

# 브라우저 끄기
browser.close()

5
10
15
20
25
30
35
40
45
50
55
60
65


In [ ]:
df1 = pd.read_csv("/Users/gyeonmunju/Desktop/PlayData/jp_company/이그니스reviews.csv")
df1

,jp_review
0,너무 급성장 했는지 아직까지도 갖춰진 체계는 없는데 뭔가를 많이 하려고 하는 회사.
1,모든게 금방 바뀌는 곳 한가지가 오래 가는 일이 없음
2,빠른 성장이 이루어지는 곳. 그러나 이를 감당할 리더의 부재.
3,브랜드 인지도 높고 희소성이 있어 대부분의 사람들이 알고 있음. 단백질 시장에서 자...
4,자율적인 분위기에 고속 성장을 이뤄나가고 있는 식품 스타트업
...,...
70,알바당시엔 굉장히 괜찮은 곳이었어요 직원분들도 다 좋아보였구요
71,자기만의 시간을 가지고 싶거나 워라밸이 중요하다면 추천하지않아요
72,겉보기와는 달리 매우 보수적인 회사. 야근을 강요하는 분위기. 그렇다고 효율적인 야...
73,랩노쉬 이미지같은 곳을 기대했다면 실망할것 투자나 외부의견에 영향을 많이 받는다


In [ ]:
from jobplanet import get_jp_reviews
get_jp_reviews("제너레잇")

{'width': 800, 'height': 600}


,긍정적 리뷰,부정적 리뷰
0,스타트업임에도 불구하고 야근이 거의 없는 편 능력있고 좋은 동료,능력있고 좋은 동료들이지만 지인으로서 좋겠지만 직장 동료로서는 단점이다. 사업의 ...
1,"20-30대로 이루어져 수평적이고 젊은 분위기이며, 서비스의 성장을 위해 모두 함...",건물 시설이 노후한 편이다. 스타트업 특성상 체계가 잡히지 않은 부분들이 있으나 만...
2,"성실함, 노력을 바탕으로 자신의 능력을 증명하면 회사가 많은 기회를 줄 수 있음. ...",마켓핏을 찾기위해 잦은 사업방향 전환이 업무의 피로도를 높일 수 있음. 사무실이 위...
3,사람간 스트레스가 거의 없는 편임. 연차를 눈치 안보고 편하게 쓸 수 있음. 호칭을...,"사무실 위치가 몽촌토성역, 잠실역 근처에 있어 접근성은 좋으나 건물이 노후화 되었음"
4,회사에 직접적인 여향을 미치는 책임감 있는 역할을 맡을 수 있다. 팀원들도 항상 ...,성장하는 스타트업이기 때문에 쳬계적이고 조직적으로 운영되는 것이 조금 어려울 때도 ...


# 테스트

In [5]:
from jobplanet import get_jp_reviews
comp = '카카오'

In [6]:
get_jp_reviews(comp)

,긍정적 리뷰,부정적 리뷰
0,비교적 자유로운 연차사용. 오래 다니다보면 하는 업무에 큰 변화 없음 다른 대기업에...,팀에 따라 사람이 별로 인 곳도 있음. 이대로 괜찮나 싶은 생각이 자주 듬. 연봉 ...
1,"복지가 매우 훌륭하고, 일이 여유로운 만큼 사람들이 모나지 않음 (상대적으로). 보...","성장이 어느정도 궤도에 올랐기에, 빡세진 않은듯. 그만큼 엄청나게 타이트한 성장을 ..."
2,네임벨류가 있어 어딜가든 사람들이 알아서 좋고 소소한 복지가 있어서 다니기 좋습니디,복지가 점점 줄고있고 무엇을 위해서 일하는지 모를때가 많습니다
3,"자유롭고 수평적인 기업 분위기 마지막 주 금요일 전사휴무, 베네핏카드 증정, 국내 ...","사내에 어시, 계약직 등 비정규직 비율이 굉장히 높다고 체감됨 최근 정규직으로의 전..."
4,평온한 문화. 시니어들이 많이 있어 도움이 됨. 시키는 일만 해도 뭐라 하지 않는 ...,루즈한 분위기. 월급 루팡 많음. 개발력을 글세... 윗사람들의 엔지니어링 능력에 ...
...,...,...
60,직원 복지와 분위기는 괜찮은 편이고 무엇보다 카카오 메신저의 시장 지배력이 워낙 굳...,경영진이 능력이나 도덕적으로 문제가 있어 보이고 누군가와의 친분이 있어야 롱런할 수...
61,"자유롭게 일할 수 있고, 좋은 사람들이 많이 모였던 팀. 프로젝트 단위별로 재미있었음.",합병 후 문화가 흐려짐. 어수선해짐. 사람은 많지만 일하는 사람이 적음.
62,복지 좋음연 한도 360 복지카드쾌적한 사무실주변에 먹을게 많음,젊은 꼰대 많음큰 기업 치고 주먹구구식 업무잦은 기획 변동대충대충 넘어가는 업무
63,비교적 수평적인 분위기실력있고 열심히 하는 동료들꽤 훌륭한 복지 수준,경영진의 도덕적 해이경영진의 도덕적 해이에 비롯한 브랜드가치 하락
